In [9]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import re

results_df = pd.read_csv("../results.csv")
# TODO ajustar dps o script
results_df.loc[results_df["plans_pct"] == 1, "plans_pct"] = 100
results_df.loc[results_df["plans_pct"] == 2, "plans_pct"] = 25
results_df.loc[results_df["plans_pct"] == 5, "plans_pct"] = 50
results_df.loc[results_df["plans_pct"] == 7, "plans_pct"] = 75

results_df.head()

,num_bdi_actions,num_rl_actions,plan_found,variation,error,score,bdi_score,complete,num_plans,plan_library_size,plans_pct,eps
0,0,100,0,21,True,3,0,False,0,25,100,117
1,26,100,1,22,False,100,100,False,6,25,100,117
2,8,94,1,23,True,3,3,True,4,25,100,117
3,34,94,1,24,False,100,100,False,8,25,100,117
4,12,90,1,25,True,3,3,True,5,25,100,117


In [10]:
plots_df = results_df[['score', 'eps', 'plans_pct']]
plots_df = plots_df.groupby(['eps', 'plans_pct']).agg(mean_score = ('score', 'mean'), max_score=('score', 'max'), min_score=('score', 'min')).reset_index()
plots_df


,eps,plans_pct,mean_score,max_score,min_score
0,117,25,13.777778,100,3
1,117,50,35.333333,100,3
2,117,75,56.888889,100,3
3,117,100,67.666667,100,3
4,173,25,13.777778,100,3
5,173,50,35.333333,100,3
6,173,75,56.888889,100,3
7,173,100,67.666667,100,3
8,231,25,13.777778,100,3
9,231,50,35.333333,100,3


In [8]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

for group, group_df in plots_df.groupby("plans_pct"):
    fig.add_trace(go.Scatter(x=group_df['eps'], y=group_df['mean_score'], name=f"Mean Score - Plan pct % {group}"))

# Edit the layout
fig.update_layout(title='BDI score',
                   xaxis_title='Episodes',
                   yaxis_title='Score')


fig.show()

In [11]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

for group, group_df in plots_df.groupby("plans_pct"):
    fig.add_trace(go.Scatter(x=group_df['eps'], y=group_df['mean_score'], name=f"Mean Score - Plan pct % {group}"))

# Edit the layout
fig.update_layout(title='BDI score',
                   xaxis_title='Episodes',
                   yaxis_title='Score')


fig.show()